In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

In [4]:
train_labels = pd.read_csv('train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
train_values = pd.read_csv('train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [5]:
train = train_values.merge(train_labels, on='building_id')
train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,2
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,3


In [6]:
train = train.drop('geo_level_1_id', axis=1)
train = train.drop('geo_level_2_id', axis=1)
train = train.drop('geo_level_3_id', axis=1)
train = train.drop('land_surface_condition', axis=1)
#train = train.drop('count_floors_pre_eq', axis=1)

In [7]:
train = train.drop('roof_type', axis=1)
train = train.drop('other_floor_type', axis=1)
train = train.drop('position', axis=1)
#train = train.drop('other_floor_type', axis=1)
train = train.drop('plan_configuration', axis=1)
train = train.drop('legal_ownership_status', axis=1)
train = train.drop('ground_floor_type', axis=1)
train = train.drop('foundation_type', axis=1)

In [8]:
train_value = train['damage_grade']
train_data = train.iloc[:, train.columns != 'damage_grade']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_data, train_value, train_size=0.7, random_state=1234)

In [26]:
knn = KNeighborsClassifier(n_neighbors=5,weights='distance',p=2,leaf_size=5,algorithm='auto',n_jobs=3)


#knn = KNeighborsClassifier(leaf_size=10, n_jobs=3, n_neighbors=4, p=3, weights='distance')


knn.fit(X_train, y_train)

KNeighborsClassifier(leaf_size=5, n_jobs=3, weights='distance')

In [27]:
predic = knn.predict(X_train)
#print(predic)

damages = pd.Series(predic).rename("damage_grade")

ids = X_train.reset_index()['building_id']

predict_train = pd.merge(ids, damages, left_index=True,right_index=True).set_index('building_id')
pred_train2 = pd.Series(predict_train.damage_grade).rename("damage_grade")

#print('Accuracy training set: {:.2f}'
     #.format(knn.score(pred_train2, y_train)))

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
precision_test2 = accuracy_score(pred_train2, y_train)

In [30]:
precision_test2

0.6907850016445565

In [31]:
predic = knn.predict(X_validation)

damages = pd.Series(predic).rename("damage_grade")

ids = X_validation.reset_index()['building_id']

predict_valid = pd.merge(ids, damages, left_index=True,right_index=True).set_index('building_id')
pred_valid = pd.Series(predict_valid.damage_grade).rename("damage_grade")

In [32]:
precision_test2 = accuracy_score(pred_valid, y_validation)

In [33]:
precision_test2

0.5389544774305778

In [34]:
pred_valid

building_id
17029     2
638130    2
123815    3
104151    2
188461    1
         ..
741937    2
788898    2
634397    3
511475    2
972228    3
Name: damage_grade, Length: 78181, dtype: uint8

In [27]:
params = {
        'p': [1,2,3,4],
        'n_neighbors': [1,2,3,4,5],
        'leaf_size': [1,5,10,15,20]
        }

In [28]:
folds = 3
param_comb = 10

In [30]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

In [33]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
random_search = RandomizedSearchCV(knn, param_distributions=params, 
        n_iter=param_comb, n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001)

In [34]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed: 15.5min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x7f019fc159e0>,
                   estimator=KNeighborsClassifier(leaf_size=5, n_jobs=3,
                                                  weights='distance'),
                   n_jobs=4,
                   param_distributions={'leaf_size': [1, 5, 10, 15, 20],
                                        'n_neighbors': [1, 2, 3, 4, 5],
                                        'p': [1, 2, 3, 4]},
                   random_state=1001, verbose=3)

In [35]:
predic = random_search.predict_proba(X_validation)

In [ ]:
print('Accuracy training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy validation set: {:.2f}'
     .format(knn.score(predic, y_validation)))

In [36]:
predic

array([[0.  , 0.75, 0.25],
       [0.  , 1.  , 0.  ],
       [0.  , 0.5 , 0.5 ],
       ...,
       [0.  , 0.5 , 0.5 ],
       [0.  , 0.5 , 0.5 ],
       [0.  , 0.25, 0.75]])

In [37]:
print(random_search.best_estimator_)

KNeighborsClassifier(leaf_size=10, n_jobs=3, n_neighbors=4, p=3,
                     weights='distance')


In [35]:
test = pd.read_csv('test_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [36]:
test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)
test = test.drop('geo_level_3_id', axis=1)
test = test.drop('land_surface_condition', axis=1)
#test = test.drop('count_floors_pre_eq', axis=1)
test = test.drop('roof_type', axis=1)
test = test.drop('other_floor_type', axis=1)
test = test.drop('position', axis=1)
test = test.drop('plan_configuration', axis=1)
test = test.drop('legal_ownership_status', axis=1)
test = test.drop('ground_floor_type', axis=1)
test = test.drop('foundation_type', axis=1)

In [37]:
predic = knn.predict(test)

damages = pd.Series(predic).rename("damage_grade")

ids = X_validation.reset_index()['building_id']

predict_test = pd.merge(ids, damages, left_index=True,right_index=True).set_index('building_id')
pred_valid = pd.Series(predict_test.damage_grade).rename("damage_grade")

In [38]:
pred_valid

building_id
17029     3
638130    3
123815    3
104151    1
188461    2
         ..
741937    3
788898    3
634397    2
511475    3
972228    2
Name: damage_grade, Length: 78181, dtype: uint8

In [43]:
ids

0         17029
1        638130
2        123815
3        104151
4        188461
          ...  
78176    741937
78177    788898
78178    634397
78179    511475
78180    972228
Name: building_id, Length: 78181, dtype: int64

In [44]:
submit = pd.merge(ids, damages, left_index=True,right_index=True).set_index('building_id')

In [45]:
submit

,damage_grade
building_id,
757764,2
139660,2
500834,2
50321,2
360305,3
...,...
53063,3
211506,2
47152,2


In [46]:
submit.to_csv('submit_knn_2.csv')